In [37]:
import numpy as np
import xtrack as xt
import xobjects as xo
import xpart as xp
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

In [38]:
#lattice parameters for ELENA at 100 keV
# from https://acc-models.web.cern.ch/acc-models/elena/scenarios/highenergy/highenergy.tfs
qx = 2.36168984503
qy = 1.38992572490
circumference = 30.40531277976 #m

# relativistic factors
gamma_rel = 1.0001066 # at 100 keV

# optics at e-cooler (approximate), in m
beta_x = 1.7
beta_y = 2.7
D_x = 1

# electron cooler parameters
current = 0.34*1e-3 # A current
length = 1 # m cooler length
radius_e_beam = 14*1e-3 #m radius of the electron beam
temp_perp = 100e-3 # <E> [eV] = kb*T
temp_long =  1e-3 # <E> [eV]
magnetic_field = 0.010 # 100 Gauss in ELENA
# idea is to study magnetic field imperfections
magnetic_field_ratio_list = [0,5e-4,1e-3,5e-3] #Iterate over different values of the magnetic field quality to see effect on cooling performance.
#magnetic_field_ratio is the ratio of transverse componenet of magnetic field and the longitudinal component. In the ideal case, the ratio is 0.

qs=0.007718714437902285
bets0=-469.32883416451523

arc_matching = xt.LineSegmentMap(
        qx=qx, qy=qy,
        dqx=0, dqy=0,
        length=circumference,
        betx=beta_x,
        bety=beta_y,
        dx=D_x,
        qs=qs,
        bets=bets0)

line_matching=xt.Line([arc_matching])
line_matching.build_tracker()



Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [54]:
# some constants, and simple computations
clight = 299792458.0
mass0 = 938.27208816*1e6 #ev/c^2

beta_rel = np.sqrt(gamma_rel**2 - 1)/gamma_rel
p0c = mass0*beta_rel*gamma_rel #eV/c
T_per_turn = circumference/(clight*beta_rel)

particle_ref= xp.Particles(mass0=mass0, q0=1, p0c=p0c)

line_matching.particle_ref = particle_ref

beta_gamma = line_matching.particle_ref._beta0*line_matching.particle_ref._gamma0

n_part = int(1e4)

beta_x_sem = 7.6 #m
beta_y_sem = 1.3 #m

# create desired beam
bunch_intensity = None
sigma_dp = 1e-3
sigma_z=-bets0*sigma_dp



eps_x= 2.25*1e-6-((D_x*sigma_dp)**2/beta_x_sem)
gemitt_x = 2.5*1e-6
gemitt_y = 2.5*1e-6
nemitt_x = gemitt_x*beta_gamma
nemitt_y = gemitt_y*beta_gamma

particles_old = xp.generate_matched_gaussian_bunch(
        num_particles=n_part,
        # total_intensity_particles=bunch_intensity,
        nemitt_x=nemitt_x, nemitt_y=nemitt_y, sigma_z = sigma_z, # in m,
        particle_ref=particle_ref ,
        line=line_matching,        
)

#particles_old.delta = np.random.normal(0, sigma_dp, n_part)
particles_old.zeta = np.random.uniform(-circumference/2, circumference/2, n_part)

# Save particles to json

with open('part.json', 'w') as fid:
    json.dump(particles_old.to_dict(), fid, cls=xo.JEncoder)

# Load particles from json file to selected context


action_x = ((particles_old.x-D_x*particles_old.delta)**2/beta_x + beta_x*particles_old.px**2)
geo_emittance_x=np.mean(action_x)/2

print('emittance:',geo_emittance_x)

action_x2 = ((particles_old.x)**2/beta_x + beta_x*particles_old.px**2)
geo_emittance_x2=np.mean(action_x2)/2

print('emittance2:',geo_emittance_x2)



emittance: 2.534392191611407e-06
emittance2: 2.8097671276783866e-06


In [55]:
data=np.load('H_angles/angles.npz')

n_steps=data['n_steps']
num_samples=data['num_samples']
delay_list=data['delay_list']
repeated_delay=data['repeated_delay']


data = np.load('results/optimize_angles.npz')

# Access the shifted_angles and filter_horizontal arrays
shifted_angles = data['shifted_angles']
filter_horizontal = data['filter_horizontal']


sorted_unique_values = np.sort(np.unique(repeated_delay))
second_min_nonzero_value = sorted_unique_values[1]
simulation_time=(np.max(repeated_delay)-second_min_nonzero_value)/1000

# angle_list=np.linspace(-40e-3,40e-3,50)

final_emittance_x=[]
final_emittance_x_normalised=[]
sigma_x_list=[]

final_emittance_y=[]
final_emittance_y_normalised=[]
sigma_y_list=[]

# simulation parameters: simulate 10 s of cooling, and take data once every 10 ms
max_time_s = 715*1e-3
int_time_s = 0.01

# compute length of simulation, as well as sample interval, in turns
num_turns = int(max_time_s/T_per_turn)
save_interval = int(int_time_s/T_per_turn)

# num_turns=50000


#plot some overall values

arc = xt.LineSegmentMap(
                qx=qx, qy=qx,
                dqx=0, dqy=0,
                length=circumference,
                betx=beta_x,
                bety=beta_y,
                dx=D_x)

beta_x_sem = 7.6 #m
beta_y_sem = 1.3 #m



for angle in tqdm(shifted_angles):

        # data = np.load(f'results/angle_x/horizontal/ELENA_angle{angle}.npz')

        # # Extract the data using the keys used when saving
        # h_delay_unique = data['h_delay_unique']
        # h_delay_unique_shifted = h_delay_unique[1:]
        # h_delay_unique_shifted=h_delay_unique_shifted-h_delay_unique_shifted[0]

        # means_h = data['means_h']
        # initial_horizontal_emittance = means_h[1]*1e-6 # because first point was faulty
        # means_sigma=data['means_sigma']

        # data = np.load(f'results/angle_x/vertical/ELENA_angle{angle}.npz')
        # means_v = data['means_v']        
        # initial_vertical_emittance = means_v[1]*1e-6 # because first point was faulty
        # # Define the whole machine
        
        electron_cooler = xt.ElectronCooler(
                length=length,
                radius_e_beam=radius_e_beam,
                current=current,
                temp_perp=temp_perp,
                temp_long=temp_long,
                magnetic_field=magnetic_field, 
                magnetic_field_ratio=1e-3,
                offset_px=angle*1e-3,
                space_charge=1)

        # create a monitor object, to reduce holded data
        monitor = xt.ParticlesMonitor(start_at_turn=0, stop_at_turn=1,
                                n_repetitions=int(num_turns/save_interval),
                                repetition_period=save_interval,
                                num_particles=n_part)

        line = xt.Line(
                elements=[monitor, electron_cooler, arc])
        line.particle_ref = particle_ref
        context = xo.ContextCpu(omp_num_threads=6)
        line.build_tracker(_context=context)

        particles=particles_old.copy()

        with open('part.json', 'r') as fid:
                particles= xp.Particles.from_dict(json.load(fid), _context=context)

        # just track all particles, and keep turn-by-turn data (memory expensive!)
        line.track(particles, num_turns=num_turns,
                turn_by_turn_monitor=False)

        # extract relevant values
        x = monitor.x[:,:,0]
        px = monitor.px[:,:,0]
        y = monitor.y[:,:,0]
        py = monitor.py[:,:,0]
        delta = monitor.delta[:,:,0]
        zeta = monitor.zeta[:,:,0]
        time = monitor.at_turn[:, 0, 0] * T_per_turn

        # compute actions. for x, remove the dp/p contribution:
        action_x = ((x-D_x*delta)**2/beta_x + beta_x*px**2)
        # for y, simple compute:
        action_y = (y**2/beta_y + beta_y*py**2)

        # compute actions. for x, remove the dp/p contribution:
        action_x2 = ((x)**2/beta_x + beta_x*px**2)
        # for y, simple compute:
        action_y2 = (y**2/beta_y + beta_y*py**2)

        norm_emittance_x=np.mean(action_x,axis=1)/2*gamma_rel*beta_rel
        norm_emittance_y=np.mean(action_y,axis=1)/2*gamma_rel*beta_rel

        geo_emittance_x=np.mean(action_x,axis=1)/2
        geo_emittance_y=np.mean(action_y,axis=1)/2

        geo_emittance_x2=np.mean(action_x2,axis=1)/2


        np.savez(f'results/optimize_angles/angle_{angle}.npz',
                
                geo_emittance_x2=geo_emittance_x2,
                geo_emittance_x=geo_emittance_x,
                geo_emittance_y=geo_emittance_y,
                norm_emittance_x=norm_emittance_x,
                norm_emittance_y=norm_emittance_y,
                x=x,  
                y=y,  
                px=px,  
                py=py,
                delta=delta,
                zeta=zeta,
                time=time)  

  0%|          | 0/51 [00:00<?, ?it/s]

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


  2%|▏         | 1/51 [00:44<36:51, 44.23s/it]

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


  4%|▍         | 2/51 [01:27<35:35, 43.57s/it]

...

 96%|█████████▌| 49/51 [35:11<01:25, 42.58s/it]

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


 98%|█████████▊| 50/51 [35:53<00:42, 42.37s/it]

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


100%|██████████| 51/51 [36:35<00:00, 43.05s/it]


In [41]:
geo_emittance_y[0]
# geo_emittance_y[0]

2.484341552125965e-06